本笔记本为[教科书](http://manipulation.csail.mit.edu/pick.html)提供了配套示例。建议将两个窗口并排打开！

In [1]:
import numpy as np
from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,  # 添加多体系统和场景图
    DiagramBuilder,               # 系统图构建器
    JacobianWrtVariable,          # 雅可比变量类型
    JointSliders,                 # 关节滑块控件
    LeafSystem,                   # 叶系统基类
    MeshcatVisualizer,            # Meshcat 可视化工具
    Parser,                       # 解析器
    StartMeshcat,                 # 启动 Meshcat
)

from manipulation import ConfigureParser, running_as_notebook  # 导入自定义解析器配置和 notebook 检测

In [3]:
# 启动可视化工具。
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


# 拣选与放置的运动学雅可比矩阵

让我们设置与上面相同的 iiwa + wsg 示例，带有滑块（但没有坐标系），这次我将展示雅可比矩阵 $J^G(q)$ 的值。

In [ ]:
class PrintJacobian(LeafSystem):
    def __init__(self, plant, frame):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._frame = frame
        self.DeclareVectorInputPort("state", plant.num_multibody_states())
        self.DeclareForcedPublishEvent(self.Publish)

    def Publish(self, context):
        state = self.get_input_port().Eval(context)
        self._plant.SetPositionsAndVelocities(self._plant_context, state)
        W = self._plant.world_frame()
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot, # 这里指定雅可比矩阵相对于关节速度，如果设置为 kV 则表示相对于速度
            self._frame,
            [0, 0, 0], # 从 frame_B 原点 Bo 指向点 Bp 的向量（在 frame_B 中表达）。
            W,  # 用于测量 Bp 速度的参考坐标系。
            W,  # 输出的速度和雅可比矩阵所表达的坐标系。
        )  # 这行是关键

        print("J_G:")
        print(np.array2string(J_G, formatter={"float": lambda x: "{:5.2f}".format(x)}))
        print(
            f"J_G 的最小奇异值: {np.min(np.linalg.svd(J_G, compute_uv=False))}"
        )
        clear_output(wait=True)


def pick_and_place_jacobians_example():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://manipulation/iiwa_and_wsg.dmd.yaml")
    plant.Finalize()

    meshcat.Delete()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph.get_query_output_port(), meshcat
    )

    G = plant.GetBodyByName("body").body_frame()
    print_jacobian = builder.AddSystem(PrintJacobian(plant, G))
    builder.Connect(plant.get_state_output_port(), print_jacobian.get_input_port())

    # 如果你想手动设置初始位置，可以使用如下代码：
    # plant.SetPositions(plant.GetMyContextFromRoot(context),
    #                   plant.GetModelInstanceByName("iiwa"),
    #                   [0, 0, 0, 0, 0, 0, 0])

    default_interactive_timeout = None if running_as_notebook else 1.0
    sliders = builder.AddSystem(JointSliders(meshcat, plant))
    diagram = builder.Build()
    sliders.Run(diagram, default_interactive_timeout)
    meshcat.DeleteAddedControls()


meshcat.DeleteAddedControls()
pick_and_place_jacobians_example()

J_G:
[[ 0.00  0.99  0.08  0.15  0.07 -0.11 -0.23  0.00  0.00]
 [ 0.00 -0.12  0.64  0.75  0.64 -0.76  0.65  0.00  0.00]
 [ 1.00  0.00  0.76 -0.64  0.76  0.64  0.72  0.00  0.00]
 [-0.62 -0.09 -0.01  0.56 -0.01 -0.16  0.00  0.00  0.00]
 [ 0.02 -0.73 -0.04 -0.06 -0.04 -0.01  0.00  0.00  0.00]
 [ 0.00  0.62  0.04  0.06  0.03 -0.04  0.00  0.00  0.00]]
J_G 的最小奇异值: 0.0013850199244189875
